# Intro

This assumes you have gone through the intro/mpi4py intro, and that you are comfortable generating ExaGO output.

Now we will go through specifics to the vizualisation frontend, how it is installed, and a typical workflow.

# Open Issues:

- npm dependencies were pinned and have security issues
- requirements.txt has pinned versions and we cannot upgrade / reproduce
- OpenAI keys and other missing steps...

# Generating file to visualize

I don't know how to do gic file option, so we are going to use OPFLOW C++ CLI to do this...

In [8]:
!opflow --version

ExaGO 1.6.0 built on Nov  9 2023


TODO: Going to let Shri fill the details in here, as likely we don't want people churning large data sets in a small demo, and we can include pre-generated data I would hope...

In [ ]:
!./opflow -netfile <netfile> -save_output -opflow_output_format JSON -gicfile <gicfilename>  

# Visualization

Launch through shell commands or with the following shell command:

In [25]:
!cd ../../viz && npm install && npm start

m##################) ⠴ reify:fsevents: sill reify mark deleted [ '/home/app/vizs
> geojson@0.1.0 prepare
> cd .. && husky install

husky - Git hooks installed

up to date, audited 1131 packages in 4s

222 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.

> geojson@0.1.0 start
> webpack-dev-server --progress --hot --open

[webpack-cli] Error: Cannot find module '../../webpack.config.local'
Require stack:
- /home/app/viz/webpack.config.js
- /home/app/viz/node_modules/webpack-cli/lib/webpack-cli.js
- /home/app/viz/node_modules/webpack-cli/lib/bootstrap.js
- /home/app/viz/node_modules/webpack-cli/bin/cli.js
- /home/app/viz/node_modules/webpack-dev-server/bin/webpack-dev-server.js
    at Module._resolveFilename (node:internal/modules/cjs/loader:1039:15)
    at Module._load (node:internal/modules/cjs/loader:885:27)
    at Module.require (node:internal/modules/cjs/loade

I seem to be stuck... Let's try and capture some more debugging information really quick.

In [40]:
!cd ../../viz/backend && sudo python3 -m pip install --upgrade pip && sudo python3 -m pip install -r requirements.txt && PYTHONPATH=/usr/local/lib/python3.11/site-packages python3 server.py

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/47/6a/453160888fab7c6a432a6e25f8afe6256d0d9f2cbd25971021da6491d899/pip-23.3.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
 * Serving Flask app 'server'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 110-927-162
127.0.0.1 - - [09/Nov/2023 02:55:53] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [09/Nov/2023 02:56:04] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [09/Nov/2023 02:56:06] "GET / HTTP/1.1" 404 -
^C


It seems like there were not only windows file endings here, but there were hard coded frozen requirements that were just for windows, and pinned for python < 3.11... Trying to run anyway just to see what happens.

Stopping here, where a few dependencies where missing, and PYTHONPATH wasn't quite correct. I am missing quite a few steps, but hopefully this is close...

In [31]:
!cd ../../viz/backend && python3 server.py

Traceback (most recent call last):
  File "/home/app/viz/backend/server.py", line 2, in <module>
    from flask import Flask
ModuleNotFoundError: No module named 'flask'



# Installation

This is done through the `postCreateCommand` that is configured in `.devcontainer/devcontainer.json` in theroy. Still debugging, so I did some stuff through Jupyter magic just to iterate quickly.

In [2]:
!cat ../../.devcontainer/devcontainer.json

{
	"build": {
		"dockerfile": "Dockerfile",
		"context": ".."
	},
	"customizations": {
		"vscode": {
			"extensions": [
				"ms-azuretools.vscode-docker",
				"github.vscode-github-actions",
				"ms-toolsai.jupyter",
				"GitHub.copilot",
				"ms-vsliveshare.vsliveshare"
			]
		}
	},
	"workspaceFolder": "/home/app/",
  	"postCreateCommand": "cd /home/app/viz && export CXX=$(which gcc) && npm install",
    "workspaceMount": "source=${localWorkspaceFolder},target=/home/app/,type=bind,consistency=cached"
}


In [3]:
!npm --version

9.2.0


In [5]:
!node --version

v18.13.0


In [7]:
!cat ../../.devcontainer/Dockerfile

# Build stage with Spack pre-installed and ready to be used
FROM spack/ubuntu-jammy as builder

# Install OS packages needed to build the software
RUN apt-get -yqq update && apt-get -yqq upgrade \
 && apt-get -yqq install autoconf \
 && rm -rf /var/lib/apt/lists/*

# What we want to install and how we want to install it
# is specified in a manifest file (spack.yaml)
RUN mkdir /opt/spack-environment \
&&  (echo spack: \
&&   echo '  specs:' \
&&   echo '  - exago@1.6.0+ipopt+python+mpi' \
&&   echo '  - petsc~fortran~hdf5~hypre+metis' \
&&   echo '  - ipopt@3.12.10+coinhsl~mumps+metis' \
&&   echo '  - coinhsl@2019.05.21+blas' \
&&   echo '  - py-jupyterlab' \
&&   echo '  - py-ipyparallel' \
&&   echo '  concretizer:' \
&&   echo '    unify: true' \
&&   echo '    reuse: true' \
&&   echo '  mirrors:' \
&&   echo '    spack: https://binaries.spack.io/develop' \
&&   echo '  packages:' \
&&   echo '    all:' \
&&   echo '      providers:' \
&&   echo '        mpi:' \
&&   echo '        